# Import necessary libraries for data manipulation, visualization, and analytics 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
!pip install mlxtend

## For advance analytics like  Import specific functions for market basket analysis and clustering

In [27]:
from mlxtend.frequent_patterns import apriori, association_rules  # for basket analysis
from sklearn.cluster import KMeans  # for RFM clustering

## load and Clean the data 

In [28]:
df = pd.read_csv('online retail.csv', encoding='utf-8-sig')

 # Print dataset size

In [ ]:
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

# Print all column names

In [ ]:
print(df.columns.tolist()) 

# Print data types for each column

In [ ]:
print(df.dtypes) 

# Convert InvoiceDate column to datetime format (dayfirst=True for UK date format)

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True)

# Remove rows with missing CustomerID and zero quantity (invalid transactions)

In [ ]:
df.dropna(subset=['CustomerID'], inplace=True)
df = df[df['Quantity'] != 0]

# Calculate revenue per transaction (Quantity * UnitPrice)

In [ ]:
df['Revenue'] = df['Quantity'] * df['UnitPrice']

In [ ]:
print(df.head())  # Display first 5 rows to check data

## Total Sales Revenue Calculation

total_revenue = df['Revenue'].sum()
print(f"Total Revenue: £{total_revenue:,.2f}")

##  Top Products Sold (By Quantity and Revenue)

In [ ]:
top_products_qty = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
top_products_rev = df.groupby('Description')['Revenue'].sum().sort_values(ascending=False).head(10)
print(top_products_qty)
print(top_products_rev)

  # Check columns again

In [ ]:
print(df.columns)

## Sales Trends Over Time (Monthly and Daily revenue)

# Ensure InvoiceDate is set as DataFrame index for resampling

In [ ]:
if df.index.name != 'InvoiceDate':
    df.set_index('InvoiceDate', inplace=True)

# Calculate monthly total revenue and plot it

In [ ]:
monthly_sales = df['Revenue'].resample('M').sum()

monthly_sales.plot(title="Monthly Sales Revenue")
plt.xlabel('Month')
plt.ylabel('Revenue (£)')
plt.tight_layout()
plt.show()

# Calculate daily total revenue and plot it

In [ ]:
daily_sales = df['Revenue'].resample('D').sum()
daily_sales.plot(title="Daily Sales Revenue")
plt.xlabel('Date')
plt.ylabel('Revenue (£)')
plt.tight_layout()
plt.show()

# Reset index after resampling

In [ ]:
df.reset_index(inplace=True)

# Debug prints to verify data structure

In [ ]:
print(df.index.name)     #  # Should be None after reset
print(df.index.dtype)    # Check index data type
print(df.columns)        # Confirm InvoiceDate is a column again

# Set snapshot date to one day after the latest InvoiceDate for RFM calculations


In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

## Customer Segmentation Using RFM (Recency, Frequency, Monetary)

In [ ]:
snapshot_date = df.index.max() + pd.Timedelta(days=1)

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'Revenue': 'sum'
})
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm = rfm[rfm['Monetary'] > 0]
print(rfm)

## Country-wise Sales Analysis

In [ ]:
country_sales = df.groupby('Country')['Revenue'].sum().sort_values(ascending=False)
print(country_sales)

## Return Rate Analysis

In [ ]:
returns = df[df['Quantity'] < 0]
return_rate = len(returns) / len(df)
print(f"Return Rate: {return_rate:.2%}")

## Market Basket Analysis
## Prepare the data in basket format (invoice-wise product matrix)

In [ ]:


basket = df[df['Country'] == 'United Kingdom']
basket = basket[basket['Quantity'] > 0]
basket = basket.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0)
basket = (basket > 0).astype(int)
print(basket.head())

## Customer Lifetime Value (CLV)

In [ ]:
clv = df.groupby('CustomerID').agg({
    'Revenue': 'sum',
    'InvoiceNo': 'nunique',
    'InvoiceDate': ['min', 'max']
})

clv.columns = ['TotalRevenue', 'Frequency', 'FirstPurchase', 'LastPurchase']
clv['CustomerAge'] = (clv['LastPurchase'] - clv['FirstPurchase']).dt.days + 1
clv['CLV'] = clv['TotalRevenue'] / clv['CustomerAge'] * 30  # Monthly value
print(clv.head())